In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import onnx
from onnx_tf.backend import prepare

In [3]:
class Net(nn.Module):    
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [4]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 1000 == 0:
            print('Train Epoch: {} \tLoss: {:.6f}'.format(
                    epoch,  loss.item()))


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the maxlog-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
        

In [5]:
train_loader = torch.utils.data.DataLoader(datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])), batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(
     datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])), batch_size=1000, shuffle=True)

9913344it [00:00, 20231491.08it/s]                             


Extracting ../data\MNIST\raw\train-images-idx3-ubyte.gz to ../data\MNIST\raw



29696it [00:00, 29669855.07it/s]         

Extracting ../data\MNIST\raw\train-labels-idx1-ubyte.gz to ../data\MNIST\raw


1649664it [00:00, 15711047.19it/s]                             

Extracting ../data\MNIST\raw\t10k-images-idx3-ubyte.gz to ../data\MNIST\raw



5120it [00:00, ?it/s]                   

Extracting ../data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ../data\MNIST\raw



In [7]:
device = torch.device("cuda")
print(device)
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)
for epoch in range(21):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader) 

cuda
Train Epoch: 0 	Loss: 2.352024

Test set: Average loss: 0.1852, Accuracy: 9492/10000 (95%)

Train Epoch: 1 	Loss: 0.553768

Test set: Average loss: 0.1192, Accuracy: 9629/10000 (96%)

Train Epoch: 2 	Loss: 0.335266

Test set: Average loss: 0.0949, Accuracy: 9704/10000 (97%)

Train Epoch: 3 	Loss: 0.152893

Test set: Average loss: 0.0813, Accuracy: 9752/10000 (98%)

Train Epoch: 4 	Loss: 0.138947

Test set: Average loss: 0.0728, Accuracy: 9763/10000 (98%)

Train Epoch: 5 	Loss: 0.243262

Test set: Average loss: 0.0682, Accuracy: 9792/10000 (98%)

Train Epoch: 6 	Loss: 0.329247

Test set: Average loss: 0.0620, Accuracy: 9801/10000 (98%)

Train Epoch: 7 	Loss: 0.089261

Test set: Average loss: 0.0572, Accuracy: 9823/10000 (98%)

Train Epoch: 8 	Loss: 0.262134

Test set: Average loss: 0.0571, Accuracy: 9830/10000 (98%)

Train Epoch: 9 	Loss: 0.100858

Test set: Average loss: 0.0522, Accuracy: 9842/10000 (98%)

Train Epoch: 10 	Loss: 0.444193

Test set: Average loss: 0.0513, Accuracy: 

In [8]:
torch.save(model.state_dict(), 'mnist.pth')

In [9]:
trained_model = Net()
trained_model.load_state_dict(torch.load('mnist.pth'))
dummy_input = Variable(torch.randn(1, 1, 28, 28))
torch.onnx.export(trained_model, dummy_input, "mnist.onnx")

In [10]:
model = onnx.load('mnist.onnx')
tf_rep = prepare(model) 

In [26]:
import numpy as np
from IPython.display import display
from PIL import Image
import matplotlib.pyplot as plt

print('Image 1:')
img = Image.open('two.png').resize((28, 28)).convert('L')
display(img)
temp = np.asarray(img, dtype=np.float32)[np.newaxis, np.newaxis, :, :]
output = model(img)
print('The digit is classified as ', np.argmax(output), 'pytorch')
output = tf_rep.run(temp)
print('The digit is classified as ', np.argmax(output), 'tf')

print('Image 2:')
img = Image.open('three.png').resize((28, 28)).convert('L')
display(img)
temp = np.asarray(img, dtype=np.float32)[np.newaxis, np.newaxis, :, :]
output = model(img)
print('The digit is classified as ', np.argmax(output), 'pytorch')
output = tf_rep.run(temp)
print('The digit is classified as ', np.argmax(output))

Image 1:


TypeError: 'ModelProto' object is not callable

In [55]:
temp = torch.randn(1, 28, 28)

output = trained_model(temp)
print(torch.argmax(output))
temp = np.asarray(temp, dtype=np.float32)[np.newaxis, :, :]
output = tf_rep.run(temp)
print(np.argmax(output))

tensor(5)
5


In [16]:
tf_rep.export_graph('mnist.pb')

INFO:tensorflow:Assets written to: mnist.pb\assets


INFO:tensorflow:Assets written to: mnist.pb\assets


In [ ]:
Net.forward()